In [1]:
import os
import sys
current_folder = os.path.dirname(os.path.abspath('__file__'))
parent_folder = os.path.dirname(current_folder)
grandparent_folder = os.path.dirname(parent_folder)
grandgrandparent_folder = os.path.dirname(grandparent_folder)
grandgrandgrandparent_folder = os.path.dirname(grandgrandparent_folder)
sys.path.append(grandgrandgrandparent_folder)

In [2]:
import optuna
from optuna.trial import TrialState
from optuna.samplers import TPESampler

import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
import copy

from methods.DeepONet import DeepONet
from methods.methodsDataset.DeepONetDataset import DeepONetDataset
from methods.DataDrivenMethods import DDMethod
from solvers.Solver import Solver

In [3]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': 0., 'nx': 101}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params_DONUT ={
             'branch': {'layer_dims': None, 'activations': None, 'device': 'cpu', 'seed': 123}, 
             'trunk':{'layer_dims': None, 'activations': None, 'device': 'cpu', 'seed': 123}
             }

In [4]:
Dmin, Dmax = 0.1, 10
D_list = np.linspace(Dmin, Dmax, 1000)

U_sols = []
solver = Solver(params={'solver': params_solver})
for d in D_list:
    solver.change_D(new_D=d)
    U_sols.append(solver.solve())

In [5]:
d_train, d_val, u_train, u_val = sklearn.model_selection.train_test_split(D_list, U_sols, test_size=0.2, 
                                                                          random_state=params_DONUT['branch']['seed'])

D_train_repeated = torch.Tensor([[d] * len(solver.x) for d in d_train]).view(-1, 1)
D_val_repeated = torch.Tensor([[d] * len(solver.x) for d in d_val]).view(-1, 1)
x = torch.Tensor(solver.x).view(-1, 1)
X_train = x.repeat(d_train.shape[0], 1)
X_val = x.repeat(d_val.shape[0], 1)

DX_train = torch.cat((D_train_repeated, X_train), dim=1)
DX_val = torch.cat((D_val_repeated, X_val), dim=1)
dU_train = torch.Tensor(np.array(u_train)).flatten()
dU_val = torch.Tensor(np.array(u_val)).flatten()

In [6]:
def get_dataset():
    return DX_train, DX_val, dU_train, dU_val

In [7]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [8]:
def define_model(trial, input_size, output_size):
    n_layers_branch = trial.suggest_int("n_layers_branch", 1, 5)
    branch_layers = [input_size]
    branch_activations = []
    
    for i in range(n_layers_branch):
        out_features = trial.suggest_int("branch_units_l{}".format(i), 52, 512)
        activation = trial.suggest_categorical("branch_activation_l{}".format(i), ["tanh", "softplus", "relu"])
        branch_layers += [out_features]
        branch_activations.append(activation)
    branch_layers += [output_size]

    n_layers_trunk = trial.suggest_int("n_layers_trunk", 1, 5)
    trunk_layers = [input_size]
    trunk_activations = []
    for i in range(n_layers_trunk):
        out_features = trial.suggest_int("trunk_units_l{}".format(i), 52, 512)
        activation = trial.suggest_categorical("trunk_activation_l{}".format(i), ["tanh", "softplus", "relu"])
        trunk_layers += [out_features]
        trunk_activations.append(activation)
    trunk_layers += [output_size]
    
    params_DONUT_trial = copy.deepcopy(params_DONUT)
    params_DONUT_trial['branch']['layer_dims'] = branch_layers
    params_DONUT_trial['branch']['activations'] = branch_activations
    params_DONUT_trial['trunk']['layer_dims'] = trunk_layers
    params_DONUT_trial['trunk']['activations'] = trunk_activations
 
    return DeepONet(params={'solver':None, 'method':params_DONUT_trial})

In [9]:
def objective(trial):
    # Generate the model.
    basis_number = trial.suggest_int("basis_number", 1, 25)
    model = define_model(trial, 1, basis_number)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    D_train, D_val, U_train, U_val = get_dataset()

    trainDataset = DeepONetDataset(x=D_train, y=U_train)
    valDataset = DeepONetDataset(x=D_val, y=U_val)

    batch_size = trial.suggest_int("batch_size", 512, 2048)
    torch.manual_seed(params_DONUT['branch']['seed'])
    trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
    valLoader = torch.utils.data.DataLoader(valDataset, batch_size=batch_size, shuffle=False)

    # Training of the model.
    loss_vals = []
    for epoch in range(1000):
        model.train()
        for i, data in enumerate(trainLoader):
            inputs, label = data
            optimizer.zero_grad()
            output = model(inputs[:, 0:1], inputs[:, 1:2])
            loss = loss_fn(output, label)
            loss.backward()
            optimizer.step()
            
        # Validation of the model.
        model.eval()
        loss_val = 0.
        with torch.no_grad():
            for i, data in enumerate(valLoader):
                inputs, label = data
                optimizer.zero_grad()
                output = model(inputs[:, 0:1], inputs[:, 1:2])
                loss_val += loss_fn(output, label).item()
        loss_val /= (i+1)
        loss_vals.append(loss_val)

        trial.report(min(loss_vals), epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return min(loss_vals)

In [10]:
sampler = TPESampler(seed=params_DONUT['branch']['seed'])
study = optuna.create_study(direction="minimize", sampler=sampler)

[I 2023-08-01 17:01:40,467] A new study created in memory with name: no-name-4c65dfee-264f-4d51-9d50-c835e491174c


In [ ]:
study.optimize(objective, n_trials=10, timeout=600)

In [ ]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
branch_activations = []
branch_layer_dims = []

trunk_activations = []
trunk_layer_dims = []

for key, value in trial.params.items():
    if key.split('_')[0] == 'branch':
        if key.split('_')[1] == 'activation':
            branch_activations.append(value)
        elif key.split('_')[1] == 'units':
            branch_layer_dims.append(value)
        
    elif key.split('_')[0] == 'trunk':
        if key.split('_')[1] == 'activation':
            trunk_activations.append(value)
        elif key.split('_')[1] == 'units':
            trunk_layer_dims.append(value)
    elif key == 'optimizer':
        optimizer = value
    elif key == 'lr':
        lr = value
    elif key == 'batch_size':
        batch_size = value
    elif key == 'basis_number':
        basis_number = value

In [ ]:
print('branch:\n', branch_activations, '\n', branch_layer_dims, 
      '\ntrunk:\n', trunk_activations, '\n', trunk_layer_dims,
      '\nhyperparameters\n',
      optimizer, '\n', lr, '\n', batch_size, '\n', basis_number)